In [1]:
import requests
import pandas as pd
from pycoingecko import CoinGeckoAPI
from google.colab import files

In [2]:
def fetch_historical_data(symbol='BTC', currency='USD', days=365):
    # CryptoCompare (OHLCV per menit)
    cc_url = f"https://min-api.cryptocompare.com/data/v2/histominute?fsym={symbol}&tsym={currency}&limit={min(days*1440, 2000)}"
    cc_data = requests.get(cc_url).json()['Data']['Data']
    df_cc = pd.DataFrame(cc_data)
    df_cc['time'] = pd.to_datetime(df_cc['time'], unit='s', utc=True)

    # CoinGecko (Metadata)
    cg = CoinGeckoAPI()
    cg_data = cg.get_coin_market_chart_by_id(
        id='bitcoin',
        vs_currency=currency.lower(),
        days=days
    )
    df_cg = pd.DataFrame({
        'time': pd.to_datetime([x[0] for x in cg_data['prices']], unit='ms', utc=True),
        'market_cap': [x[1] for x in cg_data['market_caps']],
        'total_volume': [x[1] for x in cg_data['total_volumes']]
    })

    # Gabungkan
    return pd.merge_asof(
        df_cc.sort_values('time'),
        df_cg.sort_values('time'),
        on='time',
        direction='nearest'
    ).set_index('time').resample('1S').ffill()

btc_data = fetch_historical_data(days=30)

/tmp/ipython-input-2760940924.py:27: FutureWarning: 'S' is deprecated and will be removed in a future version, please use 's' instead.
  ).set_index('time').resample('1S').ffill()


In [3]:
btc_data.head()

,high,low,open,volumefrom,volumeto,close,conversionType,conversionSymbol,market_cap,total_volume
time,,,,,,,,,,
2025-10-09 10:11:00+00:00,121886.93,121867.72,121868.28,2.342,285519.2,121869.88,direct,,2.428033e+12,5.642424e+10
2025-10-09 10:11:01+00:00,121886.93,121867.72,121868.28,2.342,285519.2,121869.88,direct,,2.428033e+12,5.642424e+10
2025-10-09 10:11:02+00:00,121886.93,121867.72,121868.28,2.342,285519.2,121869.88,direct,,2.428033e+12,5.642424e+10
2025-10-09 10:11:03+00:00,121886.93,121867.72,121868.28,2.342,285519.2,121869.88,direct,,2.428033e+12,5.642424e+10
2025-10-09 10:11:04+00:00,121886.93,121867.72,121868.28,2.342,285519.2,121869.88,direct,,2.428033e+12,5.642424e+10


In [6]:
btc_data.tail()

,high,low,open,volumefrom,volumeto,close,conversionType,conversionSymbol,market_cap,total_volume
time,,,,,,,,,,
2025-10-10 19:30:56+00:00,116386.72,116070.67,116101.45,78.79,9161562.32,116382.03,direct,,2.317643e+12,7.826314e+10
2025-10-10 19:30:57+00:00,116386.72,116070.67,116101.45,78.79,9161562.32,116382.03,direct,,2.317643e+12,7.826314e+10
2025-10-10 19:30:58+00:00,116386.72,116070.67,116101.45,78.79,9161562.32,116382.03,direct,,2.317643e+12,7.826314e+10
2025-10-10 19:30:59+00:00,116386.72,116070.67,116101.45,78.79,9161562.32,116382.03,direct,,2.317643e+12,7.826314e+10
2025-10-10 19:31:00+00:00,116443.84,116382.03,116382.03,0.00,0.00,116443.84,direct,,2.317643e+12,7.826314e+10


In [13]:
btc_data.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 120001 entries, 2025-10-09 10:11:00+00:00 to 2025-10-10 19:31:00+00:00
Freq: s
Data columns (total 10 columns):
 #   Column            Non-Null Count   Dtype  
---  ------            --------------   -----  
 0   high              120001 non-null  float64
 1   low               120001 non-null  float64
 2   open              120001 non-null  float64
 3   volumefrom        120001 non-null  float64
 4   volumeto          120001 non-null  float64
 5   close             120001 non-null  float64
 6   conversionType    120001 non-null  object 
 7   conversionSymbol  120001 non-null  object 
 8   market_cap        120001 non-null  float64
 9   total_volume      120001 non-null  float64
dtypes: float64(8), object(2)
memory usage: 10.1+ MB


In [7]:
btc_data.columns

Index(['high', 'low', 'open', 'volumefrom', 'volumeto', 'close',
       'conversionType', 'conversionSymbol', 'market_cap', 'total_volume'],
      dtype='object')

In [8]:
btc_data.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 120001 entries, 2025-10-09 10:11:00+00:00 to 2025-10-10 19:31:00+00:00
Freq: s
Data columns (total 10 columns):
 #   Column            Non-Null Count   Dtype  
---  ------            --------------   -----  
 0   high              120001 non-null  float64
 1   low               120001 non-null  float64
 2   open              120001 non-null  float64
 3   volumefrom        120001 non-null  float64
 4   volumeto          120001 non-null  float64
 5   close             120001 non-null  float64
 6   conversionType    120001 non-null  object 
 7   conversionSymbol  120001 non-null  object 
 8   market_cap        120001 non-null  float64
 9   total_volume      120001 non-null  float64
dtypes: float64(8), object(2)
memory usage: 10.1+ MB


In [9]:
btc_data.isnull().sum()

,0
high,0
low,0
open,0
volumefrom,0
volumeto,0
close,0
conversionType,0
conversionSymbol,0
market_cap,0
total_volume,0


In [10]:
btc_data.describe()

,high,low,open,volumefrom,volumeto,close,market_cap,total_volume
count,120001.000000,120001.000000,120001.000000,120001.000000,1.200010e+05,120001.000000,1.200010e+05,1.200010e+05
mean,121178.069708,121106.540374,121144.761086,25.865531,3.116529e+06,121142.019504,2.413584e+12,6.839610e+10
std,1297.300358,1319.176391,1306.223666,41.451302,4.990615e+06,1310.454321,2.609704e+10,5.159085e+09
min,116299.960000,115976.950000,116083.450000,0.000000,0.000000e+00,116083.450000,2.317643e+12,5.642424e+10
25%,121072.300000,121009.510000,121038.540000,6.681000,8.109440e+05,121038.450000,2.412483e+12,6.448917e+10
50%,121440.670000,121389.520000,121416.450000,13.710000,1.659184e+06,121416.120000,2.419925e+12,7.068922e+10
75%,121727.270000,121670.170000,121707.800000,31.080000,3.761321e+06,121707.230000,2.423445e+12,7.208826e+10
max,123818.950000,123719.390000,123762.530000,1162.430000,1.414311e+08,123762.530000,2.461600e+12,7.826314e+10


In [11]:
btc_data.shape

(120001, 10)

In [4]:
btc_data.to_csv('btc_data.csv')
files.download('btc_data.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>